In [1]:
import pytesseract
import cv2
import glob
import os

In [2]:
folder = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images/Participant 1"
images = glob.glob(folder+"/*.jpg")

In [ ]:
x, y, w, h = 300, 720 - 16, 1280 - 600, 16

for image in images:
    im = cv2.imread(image)
    # Just get the timestamp section of image
    ROI = im[y : y + h, x : x + w]
    # Invert colours
    ROI = cv2.bitwise_not(ROI)
    # Add whitespace around text (improves accuracy)
    ROI = cv2.copyMakeBorder(
        ROI, 15, 15, 0, 0, cv2.BORDER_CONSTANT, value=[255, 255, 255]
    )
    print(f"{os.path.basename(image)}: {pytesseract.image_to_string(ROI, lang='eng')}")